In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from IPython.display import update_display

# Pathfinder Playtest Encounter Creator

## Overview
This is a python based tool for DMs to design encounters for the Pathfinder Playtest Rulset. The goal is to be able to quickly and efficiently design encounters with minimal input from the user.

## Plan and Implementation
To achieve this goal, my approach is to use stepwise design in an implementation similar to the "rod-cutting" dynamic programming problem where all costs for each "length" are equal. That is, the minimum viable product goal is to naievely find all possible permutations of creatures that match our parameters.

The algorithm should take several parameters. 
- Party Level
- Party Size
- Encounter Severirty
- (Optional) The maximum number of creatures for the encounter.
- (Optional) The minimum number of creatures for the encounter.
- (Optional) An XP Budget

### The Algorithm
The algorithm itself will try to attain a combination of creatures that is equal to or under the cost of the encounter severity OR an XP budget if provided. 
To do this it will use top-down greedy programming by first choosing the largest XP cost creature that fits the budget. Once found it will store that local solution into a database and then recurse finding the next highest costing creature that fits. When a creature is found and there is still a remaning budget left, it will then use that new remaining budget and recurse down again choosing the local maximum until the base case where the remaining budget is 0. 

In [51]:
#__________This section contains all of the background setup__________

# Load the bestiary database
bestiary = pd.read_csv('data/bestiary.csv');
#bestiary

# Set the baseline encounter xp values.
BUDGETS = {'Trivial': [40,10], 'Low': [60,15], 'High': [80,20], 'Severe': [120,30], 'Extreme': [160,40]}


xp_costs = bestiary[['CREATURE NAME','10']]
nontrivial_xp_costs = xp_costs[xp_costs['10'] != '-']
pruned_xp_costs = nontrivial_xp_costs[nontrivial_xp_costs['10'] != 'X']
pruned_xp_costs
pruned_xp_costs.index[0]

145

In [58]:
# This section contains the top-down greedy algorithm.


encounter_list = []
encounter = []

# Every call to this function returns a creature name.
def find_creature(catalogue, xp, party_level):
    global encounter
    global encounter_list
    if xp == 0:
        #print("XP Budget has reached 0.")
        encounter_list.append(encounter)
        encounter = []
    else:
        for index, row in catalogue[::-1].iterrows():
            if int(row[party_level]) <= xp:
                #print("Found creature whose within budget.")
                encounter.append(row['CREATURE NAME'])
                newBudget = xp - int(row[party_level])
                find_creature(catalogue, newBudget, party_level)
            if index == catalogue.index[0] and len(encounter) > 0:
                #print("Reached End of List ", row)
                #publish the encounter
                if encounter not in encounter_list:
                    encounter_list.append(encounter)
                #remove the last item
                del encounter[-1]

def build_encounters(party_level,party_size,severity,max_c,min_c,budget):
    solutions = []
    
    # First, calcualte total encounter budget.
    if budget > 0:
        XP_Budget = budget
    else:
        if party_size > 4:
            XP_Budget = BUDGETS[severity][0] + (BUDGETS[severity][1] * (party_size - 4))
        elif party_size < 4:
            XP_Budget = BUDGETS[severity][0] - (BUDGETS[severity][1] * (4-party_size))
        else:
            XP_Budget = BUDGETS[severity][0]
    print("XP Budget: ", XP_Budget)
    
    # Next, consult the database and get the XP values for each creature based on the specified level.
    xp_costs = bestiary[['CREATURE NAME', str(party_level)]]
    nontrivial_xp_costs = xp_costs[xp_costs[str(party_level)] != '-']
    pruned_xp_costs = nontrivial_xp_costs[nontrivial_xp_costs[str(party_level)] != 'X']
    
    # Now that we have the creatures and how much they cost, build all possible encounters.
    print("Found a creature in budget: ",find_creature(pruned_xp_costs,XP_Budget,party_level))
    
    return pruned_xp_costs

#Testing Function
#Build an High threat encounter for a party of 4 level 1 adventurers with a max of five monsters and a minimum of one.
#XP Budget: 80
test = build_encounters(1,4,'High',5,1,0)
encounter_list

XP Budget:  80
Found a creature in budget:  None


[['Wight'],
 ['Werewolf fighter'],
 ['Vile necromancer'],
 ['Rust monster'],
 ['Ogre'],
 ['Minor water elemental'],
 ['Minor fire elemental'],
 ['Minor earth elemental'],
 ['Minor air elemental'],
 ['Mercenary scout'],
 ['Lion'],
 ['Hyaenodon'],
 ['Hell hound'],
 ['Grizzly bear'],
 ['Giant scorpion'],
 ['Ghast'],
 ['Gelatinous cube'],
 ['Dryad'],
 ['Drow noble cleric'],
 ['Doppelganger'],
 ['Degenerate serpentfolk'],
 ['Cockatrice'],
 ['Cleric of Rovagug'],
 ['Bunyip'],
 ['Bugbear fighter'],
 ['Ankhrav'],
 ['Animated statue'],
 ['Xulgath warrior', 'Xulgath warrior'],
 ['Xulgath warrior', 'Xulgath warrior'],
 ['Wolf'],
 ['Water mephit'],
 ['War pony'],
 ['Sewer ooze'],
 ['Riding horse'],
 ['Riding dog'],
 ['Reefclaw'],
 ['Rat swarm'],
 ['Quasit'],
 ['Plague zombie'],
 ['Orc warrior'],
 ['Lizardfolk warrior'],
 ['Lizardfolk scout'],
 ['Leopard'],
 ['Kobold scout'],
 ['Imp'],
 ['Hyena'],
 ['Hunting spider'],
 ['Hobgoblin soldier'],
 ['Goblin war chanter'],
 ['Goblin pyro'],
 ['Goblin dog'

In [13]:
# This section is the interface. 

button = widgets.Button(description="Build Encounters!")
def on_button_clicked(b):
    solution = build_encounters(party_level.value, 
                                party_size.value, 
                                severity.value,
                                creature_count.value[1], 
                                creature_count.value[0],
                                custom_budget.value)
    #print("Encounter List: ",solution)
    #type(solution)
button.on_click(on_button_clicked)

party_level = widgets.IntSlider(
    value=1,
    min=1,
    max=20,
    description='Party Level:',
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)
party_size = widgets.IntSlider(
    value=4,
    min=0,
    max=10,
    description='Party Size:',
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='40%')
)

severity = widgets.Dropdown(
    options=['Trivial', 'Low', 'High', 'Severe', 'Extreme'],
    value='Trivial',
    description='Encounter Severity: ',
    disabled=False,
    style = {'description_width': 'initial',}
)

custom_budget = widgets.BoundedIntText(
    value='0',
    min=0,
    max=500,
    step=5,
    description='Manual XP Budget (Optional):',
    disabled=False,
    style = {'description_width': 'initial'}
)

creature_count = widgets.IntRangeSlider(
    value=[1, 20],
    min=1,
    max=20,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='40%')
)


display(party_level)
display(party_size)
display(severity)
display(custom_budget)

print('')
print('')
print("Select a minimum and maximum number of creatures to use in the encounter:")
display(creature_count)





display(button)

IntSlider(value=1, description='Party Level:', layout=Layout(width='40%'), max=20, min=1, style=SliderStyle(de…

IntSlider(value=4, description='Party Size:', layout=Layout(width='40%'), max=10, style=SliderStyle(descriptio…

Dropdown(description='Encounter Severity: ', options=('Trivial', 'Low', 'High', 'Severe', 'Extreme'), style=De…

BoundedIntText(value=0, description='Manual XP Budget (Optional):', max=500, step=5, style=DescriptionStyle(de…



Select a minimum and maximum number of creatures to use in the encounter:


IntRangeSlider(value=(1, 20), continuous_update=False, layout=Layout(width='40%'), max=20, min=1)

Button(description='Build Encounters!', style=ButtonStyle())

XP Budget:  60
Found a creature in budget:  CREATURE NAME    Zombie brute
1                          60
Name: 84, dtype: object
